In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('../notebook/data/EHR.csv')
df.head(10)

In [ ]:
df= df.drop(columns=["Patient_ID"])

In [ ]:
X = df.drop(columns=["Survival_Status"])
y = df["Survival_Status"]

In [ ]:
X.head(10)

In [ ]:
y.head(10)

In [ ]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
lbl_transformer = LabelEncoder()
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [ ]:
y = lbl_transformer.fit_transform(df["Survival_Status"])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)


In [ ]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.tree import DecisionTreeClassifier
## base DTREE Model
model = DecisionTreeClassifier()
model.fit(X_train,y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("training acc",accuracy_score(y_train,y_train_pred))
print("testing acc",accuracy_score(y_test,y_test_pred))
print("score",model.score)

### hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
dtree_model = DecisionTreeClassifier()
param_grid={
    'criterion':["gini","entropy","log_loss"],
    "splitter":["best","random"],
    "max_depth":[None,2,5,10, 20, 30, 40,],
    'max_features': [None, 'sqrt', 'log2'] , # Number of features to consider when looking for the best split
    "min_samples_split":[2,4,5,8,10],
    # "min_samples_leaf":[0,1,2,3,4,5]
}
gs = GridSearchCV(estimator=dtree_model,param_grid=param_grid,cv=5,n_jobs=-1,verbose=2)
gs.fit(X_train,y_train)


In [ ]:
print("best params : " , gs.best_params_)
print("best score : " , gs.best_score_)

### use bagging

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth = 5)
model.fit(X_train,y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("training acc",accuracy_score(y_train,y_train_pred))
print("testing acc",accuracy_score(y_test,y_test_pred))
print("score",model.score)

not overfitting anymore. but accuracy is low

In [ ]:
model = RandomForestClassifier()
param_grid={
    'n_estimators':[80,100,110],
    'criterion':["gini","entropy","log_loss"],
    "max_depth":[None,2,5,8,10,15],
    'max_features': [None, 'sqrt', 'log2'] , # Number of features to consider when looking for the best split
    "min_samples_split":[2,4,5,8,10],
    "oob_score" :[True,False],
    "bootstrap" :[True,False]
}
gs = GridSearchCV(estimator=model,param_grid=param_grid,cv=5,n_jobs=-1,verbose=2)
gs.fit(X_train,y_train)

In [ ]:
gs.best_score_

In [ ]:
x = df.iloc[:,:-1]
x

In [ ]:
ohc = OneHotEncoder();
x = ohc.fit_transform(x)

In [ ]:
dfec = pd.read_csv("../notebook/data/EncodedEHRData.csv")

In [ ]:
x = dfec.iloc[:,1:]

In [ ]:
x.corr()

In [ ]:
x.cov()

In [ ]:
test_data = dfec.iloc[0]